In [1]:
import nltk



groucho_grammar = nltk.CFG.fromstring("""
    S -> NP VP
    PP -> P NP
    NP -> Det N | Det N PP | 'I'
    VP -> V NP | VP PP
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
    """)

sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)

for tree in parser.parse(sent):
    print(tree)

selected = list(parser.parse(sent))[0]



(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


In [7]:
selected = list(parser.parse(sent))[0]
# find position (in the tree) of I
for subtree in selected:
    if isinstance(subtree,str) and subtree == "I":
        # return what? from S to this word?
        print(subtree)
    else:
        for elems in subtree:
            pass

# find position (in the tree) of elephant


# recursive approach
"""
    1- traverse from S to finding e1 word
    2- traverse from S to finding e2 word
    3- compare both paths: upon some level they should coincide
    5- join the path from the point they coincide
"""

ValueError: Grammar does not cover some of the input words: "'Mary', 'saw', 'Bob'".

In [56]:
# Contexxt Free Grammar
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)
sent = "Mary saw Bob".split()
sent = "John saw Mary with a cat ".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
     print(tree)
        
tree = list(rd_parser.parse(sent))[0]
        
def findword(tree, word):
    #print("tree", type(tree),tree, tree==word)
    #print(dir(tree))
    if isinstance(tree,nltk.tree.Tree):
        result=[tree.label()]
        for stree in tree:
            subresult = findword(stree, word)
            #print("sub",subresult)
            if subresult is not None:
                result.extend(subresult)
                return result 
                break
        return None
    elif isinstance(tree,str) and tree==word:
        return []
    else:
        return None
    

path1 = findword(tree,"Mary")
path2 = findword(tree,"with")
print(path1)
print(path2)

# compare both paths
#   -> find first different element
j = 0
for i in range(1,min(len(path1),len(path2))):
    if path1[i] != path2[i]:
        j = i - 1
        break
        
# now join both list from the jth element
# we need to take into account the "order" of appearance in the tree
# left or right, which is left to the other one, cuz it's tree will be reversed
#  S VP NP Mary
#  S VP NP Bob <-> Bob Np VP S
#  always the reversed list goes first and that's it?
sublist1 = path1[j:]
print("sublist1",sublist1)
if j< len(path2)-1:
    j=j+1
sublist2 = path2[j:]
print("sublist2",sublist2)
sublist2.reverse()
print("sublist2",sublist2)
shortespath = sublist2 + sublist1
print(shortespath)

(S
  (NP John)
  (VP (V saw) (NP Mary) (PP (P with) (NP (Det a) (N cat)))))
['S', 'VP', 'NP']
['S', 'VP', 'PP', 'P']
sublist1 ['VP', 'NP']
sublist2 ['PP', 'P']
sublist2 ['P', 'PP']
['P', 'PP', 'VP', 'NP']


In [2]:
from nltk.parse.malt import MaltParser
import os
# export MALT_PARSER=$HOME/maltparser-1.8.1/
# export MALT_MODEL=$HOME/engmalt.linear-1.7.mco
#os.environ['MALT_PARSER'] = "../data/grammars/maltparser-1.8.1/"
#os.environ['MALT_MODEL'] = "../data/grammars/engmalt.poly-1.7.mco"
mp = MaltParser("../data/grammars/maltparser-1.8.1","engmalt.poly-1.7.mco")
sent1 = 'Asaf is trying to install a strange library that will never work'.split()
print(mp.parse_one(sent1).tree())

../data/grammars/maltparser-1.8.1
 path to maltparses exists
{'../data/grammars/maltparser-1.8.1/lib/liblinear-1.8.jar', '../data/grammars/maltparser-1.8.1/maltparser-1.8.1.jar', '../data/grammars/maltparser-1.8.1/lib/libsvm.jar', '../data/grammars/maltparser-1.8.1/lib/log4j.jar'}
/media/disk/home/pau/Projectes/AHLT-mai/src
(is
  Asaf
  (trying (install to (work a strange library that will never))))


In [6]:
sent1 = 'Asaf is trying to install a strange library that will never work'.split()
print(mp.parse_one(sent1).tree())

/media/disk/home/pau/Projectes/AHLT-mai/src
Asaf
(trying (install to (work a strange library that will never)))


In [22]:
# timing the dependency parsing of MaltParser
sent1 = ['Asaf is trying to install a strange library that will never work'.split(),
         'Asaf is trying to install a strange library that will never work'.split(),
         'Asaf is trying to install a strange library that will never work'.split(),
        ]
parses = mp.parse_sents(sent1)
print("done")
from time import time
start = time()
print(list(parses))
for et in parses:
    try:
        for tree in et:
            print(tree.tree())
    except:
        pass
print(time() - start)

start = time()
sent1 = 'Asaf is trying to install a strange library that will never work'.split()
print(mp.parse_one(sent1).tree())
print(time() - start)




done
/media/disk/home/pau/Projectes/AHLT-mai/src
[<list_iterator object at 0x7fd550d73e48>, <list_iterator object at 0x7fd550d73978>]
332.125369310379
/media/disk/home/pau/Projectes/AHLT-mai/src
(is
  Asaf
  (trying (install to (work a strange library that will never))))
7.6311070919036865


In [ ]:
# timing the dependency parsing of MaltParser
sent1 = ['Asaf is trying to install a strange library that will never work'.split() * 1000
        ]
parses = mp.parse_sents(sent1)
print("done")
from time import time
start = time()
print(list(parses))
numtrees = 0
for et in parses:
    try:
        for tree in et:
            if tree.tree() is not None:
                numtrees +=1
    except:
        pass
print(time() - start)

start = time()
sent1 = 'Asaf is trying to install a strange library that will never work'.split()
print(mp.parse_one(sent1).tree())
print(time() - start)